In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('india_china_google.json',lines=True)

In [3]:
df1 = pd.read_json('india_china_google_2.json',lines=True)

In [8]:
df['publish_date'][15]

{'$date': {'$numberLong': '0'}}

In [224]:
df.shape

(105, 16)

In [6]:
df1.shape

(166, 10)

In [7]:
df1.head()

,_id,channel,query,text,title,author,publish_date,summary,url,image
0,{'$oid': '5f06659002d69963de558929'},Google,india-china standoff,"Advertisement\n\nUnderstandably, due to its ev...",What Does the China-India Standoff in Ladakh M...,[Muhammad Akbar Notezai],{'$date': {'$numberLong': '0'}},,https://thediplomat.com/2020/06/what-does-the-...,https://thediplomat.com/wp-content/uploads/202...
1,{'$oid': '5f06659002d69963de55892a'},Google,india-china standoff,"After the savage clash took place on June 15, ...",India-China standoff: Chinese delayed handing ...,[],{'$date': {'$numberLong': '0'}},,https://www.businesstoday.in/current/economy-p...,https://akm-img-a-in.tosshub.com/sites/btmt/im...
2,{'$oid': '5f06659002d69963de55892b'},Google,india-china standoff,An Army convoy moves along the Srinagar-Leh hi...,"Explained: Ladakh standoff enters week 9, what...",[Sushant Singh],{'$date': {'$numberLong': '1594002280000'}},,https://indianexpress.com/article/explained/ex...,https://images.indianexpress.com/2020/06/corps...
3,{'$oid': '5f06659002d69963de55892c'},Google,india-china standoff,India to look out for signs of China not abidi...,"India China news live: Pullback on, but India ...",[],{'$date': {'$numberLong': '0'}},,https://timesofindia.indiatimes.com/india/indi...,"https://static.toiimg.com/photo/msid-76844406,..."
4,{'$oid': '5f06659002d69963de55892d'},Google,india-china standoff,"Having killed 20 of our soldiers, the statemen...","India-China standoff: While talks continue, In...",[Fe Online],{'$date': {'$numberLong': '1593423337000'}},,https://www.financialexpress.com/defence/india...,https://images.financialexpress.com/2020/06/ga...


In [8]:
def LangDet(df):
    import spacy # spacy language detector along with sample
    from spacy_langdetect import LanguageDetector
    nlp = spacy.load('en')
    nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
    langspacy=[] #detecting the complete title document into sentences
    for i in df['title']:
        ln=[]
        doc = nlp(str(i))
        for sent in doc.sents:
            try:
                ln.append(sent._.language)
            except:
                continue
        langspacy.append(ln)
    df['spacy lang'] = langspacy  # Optional
    lcode=[] 
    ldcode=[]
    for i in df['spacy lang']:
        if len(i)>1:
            ln=[]
            d=[]
            for j in range(len(i)):
                ln.append(i[j]['language'])
                d.append([i[j]['language'],i[j]['score']])
            lcode.append(ln)
            ldcode.append(d)
        else:
            lcode.append(i[0]['language'])
            ldcode.append([i[0]['language']])
    df['ldcode'] = ldcode  #Optional
    from itertools import groupby #Grouping the same languages together
    def grlist(als,n):
        result = [[k] +  [sum([j[1] for j in list(v)])/n] for k,v in groupby(sorted(als, key=lambda x: x[0]), lambda x: x[0])]
        return result
    final_l=[]  #applying the above concept on all the titles
    for i in df['ldcode']:
        if len(i)>1:
            w =grlist(i,len(i))
            w.sort(key = lambda x: x[1], reverse=True)
            final_l.append(w[0][0])
        else:
            final_l.append(i[0])
    df['final_l'] = final_l
 

In [9]:
LangDet(df)

In [10]:
LangDet(df1)

In [11]:
df.final_l.value_counts()

en    105
id      3
de      2
fr      1
so      1
it      1
sw      1
Name: final_l, dtype: int64

In [12]:
df1.final_l.value_counts()

en    159
id      4
fi      2
so      1
Name: final_l, dtype: int64

In [13]:
df = df[df['final_l'] == 'en']

In [14]:
df1 = df1[df1['final_l'] == 'en']

In [15]:
df.text.value_counts()

India and China, two nuclear-armed Asian neighbours, are in a tense diplomatic and military standoff following their first deadly border clash in more than 40 years.\n\nThe June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the Line of Actual Control (LAC), the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.\n\nMore:\n\nIndian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC, most of which remains undemarcated.\n\nThe heightened tensions between the world's two most populous countries have drawn international concerns, with the United Nations urging both sides "to exercise maximum restraint".\n\nHere are five things you need to know about the dispute:\n\nWhat happened on June 15?\n\nThe fighting on June 15 was triggered by a disagreement over two Chinese tents and observation towers that Indian officials said had b

In [16]:
df1.text.value_counts()

Never miss a great news story!\n\nGet instant notifications from Economic Times\n\nAllowNot now\n\nYou can switch off notifications anytime using browser settings.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [17]:
def op_ads(sp):
    print(len(sp))
    for i in list(sp.index):
        print(i)
        print("*", len(i))

In [18]:
import unidecode
from word2number import w2n

In [19]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lem = WordNetLemmatizer()
def text_clean(lis):
    def remove_accented_chars(text):
        """remove accented characters from text, e.g. café"""
        text = unidecode.unidecode(text)
        return text
    lis['txt'] = lis['0'].apply(lambda x:remove_accented_chars(x))
    def wor2no(text):
        try:
            doc = nlp(text)
            tokens = [w2n.word_to_num(token.text) if token.pos_ == 'NUM' else token for token in doc]
            return ' '.join(tokens)
        except:
            return text
    lis['txt'] = lis['txt'].apply(lambda x:wor2no(x))
    lis['txt'] = lis['txt'].apply(lambda x: tokenizer.tokenize(x.lower()))
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    lis['txt_stop'] = lis['txt'].apply(lambda x: [word for word in x if word not in stop_words])
    lis['txt_stop'] = lis['txt_stop'].apply(lambda x: " ".join(x))
    def  lemm_txt(sent):
        tokens = word_tokenize(sent.lower())
        lemme = [lem.lemmatize(term) for term in tokens]
        res = " ".join(lemme)
        return res
    lis['txt_lem'] = lis['txt_stop'].apply(lambda x: lemm_txt(x))
    return lis

In [20]:
def ad_handle(df,n):
    ad = []
    df['small_size'] = df['text'].apply(lambda x:len(x)<=n)  # Getting a column that categorises if length is >500 in bool
    sdf = df[df['small_size']==True] # Creating a DataFrame considering only the articles with content size less than 500
    sp = sdf.text.value_counts()    # Getting an output of frequency count of all the text content among above DataFrame
    oni=sp[sp>1]                    # Considering only the repeating text i.e freq is more than one 
    print('The total no of articles are: ',df.shape[0])
    print('The total no of small articles with content size less than {} are: '.format(n),sdf.shape[0], 'i.e {}% of total'.format((sdf.shape[0]/df.shape[0])*100))
    print('The total no of repeating small articles are: ',sum(oni), 'i.e {}% of total'.format((sum(oni)/df.shape[0])*100))
    ads = pd.DataFrame(list(oni.index), columns=['0'])
#    ads.head()
    op_ads(oni)
    ads_clean = text_clean(ads)
    ads_clean.to_csv('Ads/Ads.csv')
    print(ads_clean['txt_lem'].values)
    for i in ads_clean['txt_lem'].values:
        ad.extend(i.split())
    ad = set(ad)
    print(ad)

In [21]:
ad_handle(df,400)

The total no of articles are:  105
The total no of small articles with content size less than 400 are:  7 i.e 6.666666666666667% of total
The total no of repeating small articles are:  2 i.e 1.9047619047619049% of total
1
Never miss a great news story!

Get instant notifications from Economic Times

AllowNot now

You can switch off notifications anytime using browser settings.
* 157
['never miss great news story get instant notification economic time allownot switch notification anytime using browser setting']
{'switch', 'browser', 'setting', 'get', 'allownot', 'instant', 'news', 'great', 'never', 'time', 'anytime', 'miss', 'using', 'notification', 'story', 'economic'}


In [22]:
print(df['text'][0])

India and China, two nuclear-armed Asian neighbours, are in a tense diplomatic and military standoff following their first deadly border clash in more than 40 years.

The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the Line of Actual Control (LAC), the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.

More:

Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC, most of which remains undemarcated.

The heightened tensions between the world's two most populous countries have drawn international concerns, with the United Nations urging both sides "to exercise maximum restraint".

Here are five things you need to know about the dispute:

What happened on June 15?

The fighting on June 15 was triggered by a disagreement over two Chinese tents and observation towers that Indian officials said had been built on i

In [23]:
import torch
try:
    import transformers
    from transformers import BartTokenizer, BartForConditionalGeneration
except ImportError:
    raise ImportError(INSTALL_MSG)
from IPython.display import display, Markdown
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [24]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
def bartsum(sents):
    article_input_ids = tokenizer.batch_encode_plus([sents], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
    summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=142,
                             min_len=25,
                             no_repeat_ngram_size=3)

    summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    display(Markdown('> **abstractive-Summary: **'+summary_txt))

In [25]:
bartsum(df['text'][0])

> **abstractive-Summary: **Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC. The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.

In [26]:
len('Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC. The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.'.split())

56

In [27]:
len('Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC. The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.')

337

In [28]:
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

article_input_ids = tokenizer.batch_encode_plus([df['text'][0]], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=80,
                             min_len=20,
                             no_repeat_ngram_size=3)

summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: **'+summary_txt))

> **Summary: **Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC. The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.

In [29]:
article_input_ids = tokenizer.batch_encode_plus([df['text'][0]], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=200,
                             min_len=20,
                             no_repeat_ngram_size=3)

summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: **'+summary_txt))

> **Summary: **Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC. The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.

In [30]:
article_input_ids = tokenizer.batch_encode_plus([df['text'][0]], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=500,
                             min_len=20,
                             no_repeat_ngram_size=3)

summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: **'+summary_txt))

> **Summary: **Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC. The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.

In [31]:
article_input_ids = tokenizer.batch_encode_plus([df['text'][0]], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=400,
                             min_len=200,
                             no_repeat_ngram_size=3)

summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary: **'+summary_txt))

> **Summary: **Indian and Chinese troops have been engaged in the standoff since early May at several points along the 3,500km (2,200-mile) LAC. The June 15 incident in the disputed Galwan Valley, an arid Himalayan area along the de facto border between the two nations, left 20 Indian soldiers dead. China has yet to officially declare its casualties.

In [32]:
#!pip install tensorflow-hub

In [33]:
import tensorflow_hub as hub
import tensorflow as tf

In [34]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

INFO:absl:Using C:\Users\ADMINI~1\AppData\Local\Temp\tfhub_modules to cache modules.


In [35]:
sample_statement = ["Jack while talking over the cell phone entered the prison cell to extract blood cell samples of Jill and made an entry in the excel cell about the blood sample collection."]
# Extract ELMo features
embeddings = elmo(sample_statement, signature="default", as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [36]:
print(embeddings)

Tensor("module_apply_default/aggregation/mul_3:0", shape=(1, 31, 1024), dtype=float32)


In [37]:
def ELMoEmbedding(input_text):
    return elmo(tf.reshape(tf.cast(input_text, tf.string), [-1]), signature="default", as_dict=True)["elmo"]

In [38]:
ELMoEmbedding(sample_statement)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<tf.Tensor 'module_apply_default_1/aggregation/mul_3:0' shape=(1, 31, 1024) dtype=float32>

In [39]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

# change this to read in your data
finder = BigramCollocationFinder.from_words(
   nltk.corpus.genesis.words('english-web.txt'))

# only bigrams that appear 3+ times
finder.apply_freq_filter(3) 

# return the 5 n-grams with the highest PMI
finder.nbest(bigram_measures.pmi, 5)  

[('Beer', 'Lahai'),
 ('Lahai', 'Roi'),
 ('gray', 'hairs'),
 ('Most', 'High'),
 ('ewe', 'lambs')]

In [44]:
nltk.corpus.genesis.words('english-web.txt')

['In', 'the', 'beginning', 'God', 'created', 'the', ...]

In [51]:
df['text'][5]

'Indian and Chinese troops along the Line of Actual Control. | File image | Photo Credit: IANS\n\nKey Highlights Several countries have extended solidarity with India and expressed their angst over the impasse with China, with the United States and Germany reaffirming their strong ties with India in these trying times The think tank observed that the LAC attack led to Chinaâ€™s most serious security situation with India in recent years, and flared emotions have meant that the potential for rapid escalation is rife\n\nAmsterdam: The mood around the Ladakh standoff and the subsequent deaths of 20 brave Indian Army soldiers in a clash with Chinese Peopleâ€™s Liberation Army on the intervening night of June 15-16 has left the bilateral ties between the two countries in tatters.\n\nIndiaâ€™s long-standing position on the importance of healthy diplomatic ties with China was compromised with after Chinese troops made continued advancements towards the LAC since early May, leading to the horro

In [104]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

# change this to read in your data
finder = BigramCollocationFinder.from_words(df['text'][5].split())
# only bigrams that appear 3+ times
finder.apply_freq_filter(3) 

# return the 5 n-grams with the highest PMI
end = finder.nbest(bigram_measures.pmi, 10)

In [105]:
end

[('United', 'States'),
 ('most', 'serious'),
 ('South', 'China'),
 ('Chinese', 'troops'),
 ('with', 'India'),
 ('ties', 'with'),
 ('in', 'recent'),
 ('Line', 'of'),
 ('of', 'Actual'),
 ('India', 'in')]

In [106]:
nltk.pos_tag(['inspace'])

[('inspace', 'NN')]

j = ' '.join(i)
    jo = nltk.pos_tag([j])
    print(jo)
    if (jo[0][1] == 'NNP') | (jo[0][1] == 'NNPS') | (jo[0][1] == 'NN') | (jo[0][1] == 'NNS'):
        elo.append(jo)

In [113]:
elo =[]
for i in end:
    check = 0
    on = nltk.pos_tag(i)
    print(on)
    if (on[0][1] == 'NNP') | (on[0][1] == 'NNPS') | (on[0][1] == 'NN') | (on[0][1] == 'NNS'):
        elo.append(on[0])
        check += 1
    if (on[1][1] == 'NNP') | (on[1][1] == 'NNPS') | (on[1][1] == 'NN') | (on[1][1] == 'NNS'):
        elo.append(on[1])
        check += 1
    if check ==2:
        j = ' '.join(i)
        jo = nltk.pos_tag([j])
        elo.append(jo[0])

[('United', 'NNP'), ('States', 'NNPS')]
[('most', 'RBS'), ('serious', 'JJ')]
[('South', 'NNP'), ('China', 'NNP')]
[('Chinese', 'JJ'), ('troops', 'NNS')]
[('with', 'IN'), ('India', 'NNP')]
[('ties', 'NNS'), ('with', 'IN')]
[('in', 'IN'), ('recent', 'JJ')]
[('Line', 'NN'), ('of', 'IN')]
[('of', 'IN'), ('Actual', 'NNP')]
[('India', 'NNP'), ('in', 'IN')]


In [119]:
ik = []
for i in elo:
    ik.append(i[0])

In [120]:
ik

['United',
 'States',
 'United States',
 'South',
 'China',
 'South China',
 'troops',
 'India',
 'ties',
 'Line',
 'Actual',
 'India']

In [65]:
' '.join(end[0])

'United States'

In [149]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
def Tags(text):
    # change this to read in your data
    finder = BigramCollocationFinder.from_words(text.split())
    # only bigrams that appear 3+ times
    finder.apply_freq_filter(3)
    # return the 5 n-grams with the highest PMI
    end= finder.nbest(bigram_measures.pmi, 10)  
    elo =[]
    for i in end:
        on = nltk.pos_tag(i)
        #print(on)
        if ((on[0][1] == 'NNP') | (on[0][1] == 'NNPS') | (on[0][1] == 'NN') | (on[0][1] == 'NNS'))&((on[1][1] == 'NNP') | (on[1][1] == 'NNPS') | (on[1][1] == 'NN') | (on[1][1] == 'NNS')):
            j = ' '.join(i)
            jo = nltk.pos_tag([j])
            elo.append(jo[0])
        else:
            if (on[0][1] == 'NNP') | (on[0][1] == 'NNPS') | (on[0][1] == 'NN') | (on[0][1] == 'NNS'):
                elo.append(on[0])
            if (on[1][1] == 'NNP') | (on[1][1] == 'NNPS') | (on[1][1] == 'NN') | (on[1][1] == 'NNS'):
                elo.append(on[1])
    ik = []
    for i in elo:
        ik.append(i[0])
    return ik

In [150]:
df['tags'] = df['text'].apply(lambda x:Tags(x))

In [151]:
df['tags'] = df['tags'].apply(lambda x: list(set(x)))

In [155]:
 df['tags']

0      [June, India, Galwan Valley, troops, soldiers,...
1          [sides, June, side, Galwan Valley, consensus]
2                [sides, Pangong Tso, situation, troops]
3      [border standoff, Doklam, Army, Galwan Valley,...
4                     [June, PLA, Galwan, Galwan Valley]
                             ...                        
109           [Actual, countries, soldiers, Line, China]
110                                                   []
111                      [South China, China Sea, China]
112    [sides, Aksai Chin, India, Galwan Valley, Galw...
113    [interest, Image Credit:, Valley, Galwan Valle...
Name: tags, Length: 105, dtype: object

In [134]:
#df['tags'].to_csv('tags.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


for i in df['tags']:
    for j in i:
        

In [156]:
tag_list=[]
for i in df['tags']:
    for j in (i):
        tag_list.append(j)
from collections import Counter
cnt=Counter(tag_list)
taglists = cnt.most_common(10)

In [223]:
taglists

[('India', 22),
 ('troops', 18),
 ('Galwan Valley', 17),
 ('Line', 15),
 ('China', 15),
 ('June', 14),
 ('Actual', 12),
 ('soldiers', 11),
 ('sides', 10),
 ('Control', 10)]

In [173]:
tag_weights =[]
for i in range(len(taglists)):
    tot = sum(n for _, n in taglists)
    tag_weights.append((taglists[i][0],round(taglists[i][1]/tot,4)))

In [174]:
tag_weights

[('India', 0.1528),
 ('troops', 0.125),
 ('Galwan Valley', 0.1181),
 ('Line', 0.1042),
 ('China', 0.1042),
 ('June', 0.0972),
 ('Actual', 0.0833),
 ('soldiers', 0.0764),
 ('sides', 0.0694),
 ('Control', 0.0694)]

In [1]:
keywords = [x[0] for x in tag_weights]

NameError: name 'tag_weights' is not defined

In [177]:
score= []
def word_score(text,wts):
    words = text.split()
    w_sc = []
    for i in wts:
        cnt = 0
        for word in words:
            if word == i[0]:
                cnt+=1
        word_score = cnt*i[1]
        w_sc.append(word_score)
    return sum(w_sc)    

In [178]:
word_score(df['text'][0],tag_weights)

5.208699999999999

In [179]:
df['score'] = df['text'].apply(lambda x: word_score(x,tag_weights))

In [191]:
df

,_id,channel,query,text,title,author,publish_date,summary,url,image,spacy lang,ldcode,final_l,small_size,tags,score
0,{'$oid': '5f07a5765b177cd19f969f10'},Google,india china standoff LAC,"India and China, two nuclear-armed Asian neigh...",Five things to know about the India-China bord...,[],{'$date': {'$numberLong': '0'}},,https://www.aljazeera.com/news/2020/06/india-c...,https://www.aljazeera.com/mritems/Images/2020/...,"[{'language': 'en', 'score': 0.9999973328097994}]",[en],en,False,"[June, India, Galwan Valley, troops, soldiers,...",5.2087
1,{'$oid': '5f07a5765b177cd19f969f11'},Google,india china standoff LAC,"india\n\nUpdated: Jun 26, 2020 00:22 IST\n\nRe...","LAC standoff: Hope India meets China halfway, ...","[Press Trust Of India, Posted Niyati Singh]",{'$date': {'$numberLong': '1593111092000'}},,https://www.hindustantimes.com/india-news/lac-...,https://www.hindustantimes.com/rf/image_size_9...,"[{'language': 'en', 'score': 0.999994921823019...","[[en, 0.9999949218230191], [nl, 0.999994213254...",en,False,"[sides, June, side, Galwan Valley, consensus]",4.5834
2,{'$oid': '5f07a5765b177cd19f969f12'},Google,india china standoff LAC,Chinese troops have killed at least 20 Indian ...,India-China standoff: How the situation sudden...,[],{'$date': {'$numberLong': '1592318580000'}},,https://m.economictimes.com/news/defence/india...,"https://img.etimg.com/thumb/msid-76402346,widt...","[{'language': 'en', 'score': 0.999997027463945...","[[en, 0.9999970274639454], [en, 0.999999359947...",en,False,"[sides, Pangong Tso, situation, troops]",1.2985
3,{'$oid': '5f07a5765b177cd19f969f13'},Google,india china standoff LAC,Image Source : INDIA TV India-China Faceoff: T...,India-China LAC Standoff: 1975 Tulung LA to Ga...,"[New Delhi, Image Source, India Tv]",{'$date': {'$numberLong': '1592373956000'}},,https://www.indiatvnews.com/news/india/india-c...,https://resize.indiatvnews.com/en/resize/newbu...,"[{'language': 'en', 'score': 0.9999970702589753}]",[en],en,False,"[border standoff, Doklam, Army, Galwan Valley,...",3.3266
4,{'$oid': '5f07a5765b177cd19f969f14'},Google,india china standoff LAC,"india\n\nUpdated: Jun 30, 2020 02:39 IST\n\nIn...","Indian, Chinese armies set for Ladakh meeting",[Rahul Singh],{'$date': {'$numberLong': '1593433198000'}},,https://www.hindustantimes.com/india-news/indi...,https://www.hindustantimes.com/rf/image_size_9...,"[{'language': 'en', 'score': 0.8571366217424891}]",[en],en,False,"[June, PLA, Galwan, Galwan Valley]",1.7221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,{'$oid': '5f07a5765b177cd19f969f7d'},Google,Galwan dispute,China has built new structures near the site o...,Galwan Valley: Satellite images 'show China st...,[],{'$date': {'$numberLong': '0'}},,https://www.bbc.com/news/world-asia-53174887,https://ichef.bbci.co.uk/news/1024/branded_new...,"[{'language': 'cy', 'score': 0.857140712349764...","[[cy, 0.857140712349764], [en, 0.9999923977934...",en,False,"[Actual, countries, soldiers, Line, China]",3.9030
110,{'$oid': '5f07a5765b177cd19f969f7e'},Google,Galwan dispute,The Galwan River flows from the disputed Aksai...,Galwan River,[],{'$date': {'$numberLong': '0'}},,https://en.wikipedia.org/wiki/Galwan_River,https://upload.wikimedia.org/wikipedia/commons...,"[{'language': 'en', 'score': 0.9999955923713719}]",[en],en,False,[],1.4794
111,{'$oid': '5f07a5765b177cd19f969f7f'},Google,Galwan dispute,"india\n\nUpdated: Jun 20, 2020 13:39 IST\n\nMi...","Not just Indiaâ€™s Galwan, China has a long li...",[Shishir Gupta],{'$date': {'$numberLong': '1592635121000'}},,https://www.hindustantimes.com/india-news/not-...,https://www.hindustantimes.com/rf/image_size_9...,"[{'language': 'en', 'score': 0.8571417317229016}]",[en],en,False,"[South China, China Sea, China]",3.7092
112,{'$oid': '5f07a5765b177cd19f969f80'},Google,Galwan dispute,"Without firing a single shot, Indian and Chine...",Why India-China border clash in Galwan Valley ...,"[Marcelo Duhalde, Dennis Wong, Kaliz Lee, Http...",{'$d

In [183]:
scored_df = df.sort_values(by = 'score', ascending=False)

In [185]:
scored_df.drop_duplicates(subset ="text", keep = 'first', inplace = True) 

In [192]:
top30 = scored_df.head(30)['text']

In [188]:
top10 = scored_df.head(10)['text']

In [189]:
top10

64     Diplomatic relations between the People's Repu...
15     2020 Chinaâ€“India border standoff\n\nThe 2020...
36     Some companies have already developed strategi...
80     India-China dispute Satellite images suggest C...
39     Entity speculated to become a superpower\n\nOn...
17     There have been many different reasons attribu...
87     Japan Backs India On Ladakh, "Opposes" Change ...
94     Fast News\n\nStand-off between Indian and Chin...
106    Here are the latest developments:\n\nMinisters...
102    The trajectory of the India-China crisis begin...
Name: text, dtype: object

In [193]:
top30

64     Diplomatic relations between the People's Repu...
15     2020 Chinaâ€“India border standoff\n\nThe 2020...
36     Some companies have already developed strategi...
80     India-China dispute Satellite images suggest C...
39     Entity speculated to become a superpower\n\nOn...
17     There have been many different reasons attribu...
87     Japan Backs India On Ladakh, "Opposes" Change ...
94     Fast News\n\nStand-off between Indian and Chin...
106    Here are the latest developments:\n\nMinisters...
102    The trajectory of the India-China crisis begin...
93     LAC Border News: India & China got engaged int...
0      India and China, two nuclear-armed Asian neigh...
1      india\n\nUpdated: Jun 26, 2020 00:22 IST\n\nRe...
76     A few possible reasons have been put forward t...
84     The ongoing military standoffs with China at m...
86     People stand by the banks of the Pangong Lake,...
12     New Delhi: India on Thursday said that the key...
112    Without firing a single 

In [190]:
for i in top10:
    bartsum(i)

> **abstractive-Summary: **China-India relations refers to the bilateral relationship between the People's Republic of China and the Republic of India. The modern relationship began in 1950 when India was among the first countries to recognize China as the legitimate government of Mainland China. Relations have been characterised by border disputes, resulting in three military conflicts. Since the late 1980s, both countries have successfully rebuilt diplomatic and economic ties.

> **abstractive-Summary: **The 2020 Chinaâ€“India skirmishes are part of an ongoing military standoff between China and India. Since 5 May 2020, Chinese and Indian troops have reportedly engaged in aggressive melee, face-offs and skirmishes at locations along the Sino-Indian border. Experts have postulated that the standoffs are Chinese pre-emptive measures in responding to the Darbukâ’Shyokâ€ “DBO Road project in Ladakh.

> **abstractive-Summary: **China and India are burying the hatchet after four-plus decades of hostility. Some companies have already developed strategies that make use of both countries' capabilities. The two nations are also reviving their old cultural and religious ties. Their desire to strike a partnership is evident: High-level visits often take place between them.

> **abstractive-Summary: **Satellite images suggest Chinese activity at India's Himalayan border before clash. China brought in pieces of machinery, cut a trail into a Himalayan mountainside and may have even dammed a river. The fighting was triggered by a row over two Chinese tents and observation towers that India said had been built on its side of the Line of Actual Control.

> **abstractive-Summary: **Currently, only the United States fulfills the criteria to be considered a superpower. China has been referred to as an emerging superpower, given that Beijing's power is now beyond the classification of a Great Power. The European Union[5] and the emerging BRIC economies comprising Brazil, Russia and India are most commonly described as potential superpowers.

> **abstractive-Summary: **Hemant Adlakha is a professor at JNU's Centre for Chinese and South East Asian Studies. He says the change in Chinese thought towards India came after 2017â€™s Doklam standoff. In Chinese media, all the blame and reasons are being attributed to the Indian forces, he says. The strategy on India for a long time is that somehow India has to be kept under control.

> **abstractive-Summary: **India has decided to increase its strength along 3,488-km of Line of Actual control LAC -- the de facto border with China. Delhi Chief Minister Arvind Kejriwal says India will win even though it is "fighting China on two fronts" China emphasised today that there was no need for the intervention of a "third party" to resolve its current standoff with India.

> **abstractive-Summary: **In this study, we look at the factors that determine whether or not a person will be able to take part in the contest. We look at how each factor affects the outcome of the game. We also look at what factors determine whether the winner will be female or male. We then compare the results to see which factors determine the winner.

> **abstractive-Summary: **India and China will hold a third round of Major-General level talks on Friday following the release of ten Indian soldiers held captive by the Peopleâ€™s Liberation Army (PLA) The talks are being held in the aftermath of June 15 violent clash between border troops at Galwan Valley, Ladakh, which lead to the death of 20 Indian soldiers. Congress leader Rahul Gandhi on Friday accused senior ministers in the government of "lying" to protect the prime minister.

> **abstractive-Summary: **The India-China border issue is no longer mired in Nehruâ€™s legal-historical approach. The issue moved to the political plane, where it has since rested in peace. There are at least four claim lines in the Eastern sector. In the Western sector there are three varying claim lines.

In [194]:
for i in top30:
    bartsum(i)

> **abstractive-Summary: **China-India relations refers to the bilateral relationship between the People's Republic of China and the Republic of India. The modern relationship began in 1950 when India was among the first countries to recognize China as the legitimate government of Mainland China. Relations have been characterised by border disputes, resulting in three military conflicts. Since the late 1980s, both countries have successfully rebuilt diplomatic and economic ties.

> **abstractive-Summary: **The 2020 Chinaâ€“India skirmishes are part of an ongoing military standoff between China and India. Since 5 May 2020, Chinese and Indian troops have reportedly engaged in aggressive melee, face-offs and skirmishes at locations along the Sino-Indian border. Experts have postulated that the standoffs are Chinese pre-emptive measures in responding to the Darbukâ’Shyokâ€ “DBO Road project in Ladakh.

> **abstractive-Summary: **China and India are burying the hatchet after four-plus decades of hostility. Some companies have already developed strategies that make use of both countries' capabilities. The two nations are also reviving their old cultural and religious ties. Their desire to strike a partnership is evident: High-level visits often take place between them.

> **abstractive-Summary: **Satellite images suggest Chinese activity at India's Himalayan border before clash. China brought in pieces of machinery, cut a trail into a Himalayan mountainside and may have even dammed a river. The fighting was triggered by a row over two Chinese tents and observation towers that India said had been built on its side of the Line of Actual Control.

> **abstractive-Summary: **Currently, only the United States fulfills the criteria to be considered a superpower. China has been referred to as an emerging superpower, given that Beijing's power is now beyond the classification of a Great Power. The European Union[5] and the emerging BRIC economies comprising Brazil, Russia and India are most commonly described as potential superpowers.

...

> **abstractive-Summary: **China says the Galwan Valley where Chinese and Indian troops were engaged in a deadly clash falls entirely within its territory. Beijing blamed New Delhi for the June 15 military clash at the border that left 20 Indian soldiers dead. Indian and Chinese troops have been engaged in an eyeball-to-eyeball standoff at several points on their 3,500km (2,200-mile) border.

> **abstractive-Summary: **Twenty Indian Army bravehearts, including a Colonel of 16 Bihar Regiment, were killed in the clash with Chinese troops in eastern Ladakh's Galwan Valley. The Army initially said on Tuesday that an officer and two soldiers were killed. But in a late evening statement, it revised the figure to 20 saying 17 others who "were critically injured in the line of duty" The news was met with wide anti-China protest in India.

> **abstractive-Summary: **India and China share a 3,440km long border. Both countries have overlapping territorial claims. The sudden interest in Galwan Valley might well be strategic. It lies close to Aksai Chin, a disputed area (claimed by India) but effectively under the Chinese control.

> **abstractive-Summary: **India-China armies hold 3rd Corps Commander-level talks in Chushul. India deployed six T-90 missile-firing tanks, and top-of-the-line shoulder-fired anti-tank missile systems. New satellite images from Line of Actual Control (LAC) in the Pangong Tso area shows China has expanded its heliport located 21.3 km northeast from LAC.

> **abstractive-Summary: **This was their first meeting after the first of two-and-one-year-old, the second is the third of the first, the fourth is the fifth, the sixth and the sixth is the sixth, the seventh is the eighth, the tenth, or the tenth is the tenth or the first is the first or the sixth of the sixth or the eighth of the tenth to be the first to be a “vast’s” of the “one-to-one” “ ’“”   “ ’A’ ‘’, ””,

## corona summary

In [195]:
data  = pd.read_json('full_corona.json',lines=True)

In [198]:
corona = data[data['publish_date'] == '2020-04-03 00:00:00']

In [200]:
import nltk
from nltk.collocations import *
def tag_scorer(df):

    bigram_measures = nltk.collocations.BigramAssocMeasures()
    def Tags(text):
        # change this to read in your data
        finder = BigramCollocationFinder.from_words(text.split())
        # only bigrams that appear 3+ times
        finder.apply_freq_filter(3)
        # return the 5 n-grams with the highest PMI
        end= finder.nbest(bigram_measures.pmi, 10)  
        elo =[]
        for i in end:
            on = nltk.pos_tag(i)
            #print(on)
            if ((on[0][1] == 'NNP') | (on[0][1] == 'NNPS') | (on[0][1] == 'NN') | (on[0][1] == 'NNS'))&((on[1][1] == 'NNP') | (on[1][1] == 'NNPS') | (on[1][1] == 'NN') | (on[1][1] == 'NNS')):
                j = ' '.join(i)
                jo = nltk.pos_tag([j])
                elo.append(jo[0])
            else:
                if (on[0][1] == 'NNP') | (on[0][1] == 'NNPS') | (on[0][1] == 'NN') | (on[0][1] == 'NNS'):
                    elo.append(on[0])
                if (on[1][1] == 'NNP') | (on[1][1] == 'NNPS') | (on[1][1] == 'NN') | (on[1][1] == 'NNS'):
                    elo.append(on[1])
        ik = []
        for i in elo:
            ik.append(i[0])
        return ik
    df['tags'] = df['text'].apply(lambda x:Tags(x))
    df['tags'] = df['tags'].apply(lambda x: list(set(x)))
    tag_list=[]
    for i in df['tags']:
        for j in (i):
            tag_list.append(j)
    from collections import Counter
    cnt=Counter(tag_list)
    taglists = cnt.most_common(10)
    tag_weights =[]
    for i in range(len(taglists)):
        tot = sum(n for _, n in taglists)
        tag_weights.append((taglists[i][0],round(taglists[i][1]/tot,4)))
    score= []
    def word_score(text,wts):
        words = text.split()
        w_sc = []
        for i in wts:
            cnt = 0
            for word in words:
                if word == i[0]:
                    cnt+=1
            word_score = cnt*i[1]
            w_sc.append(word_score)
        return sum(w_sc)    
    df['score'] = df['text'].apply(lambda x: word_score(x,tag_weights))
    scored_df = df.sort_values(by = 'score', ascending=False)
    scored_df.drop_duplicates(subset ="text", keep = 'first', inplace = True)
    top30 = scored_df.head(30)['text']
    return top30

In [201]:
tag_scorer(corona)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

589     Alabama System to Open $250M Lines of Credit\n...
517     â€œI just really want people to understand tha...
793     Last weekend, the coronavirus took the life of...
514     (CNN) Kevin and Ryan Sheehan will never know w...
641     Tokyo (CNN) Each day seems to bring more bad n...
601     The Centers for Disease Control and Prevention...
622     New Delhi (CNN) One of Asia's biggest slums ha...
637     New Delhi (CNN) One of Asia's biggest slums ha...
511     What are the symptoms caused by the Covid-19 v...
575     Maria D Van Kerkhove, World Health Organizatio...
507     Public health experts are increasingly worried...
569     Federal emergency officials said they were exh...
541     (CNN) With more than 6,000 deaths from coronav...
767     As soon as Alison Galvani learned of the COVID...
529     Study design\n\nParticipants were recruited ye...
666     But the problem is not the animals, according ...
626     Sacramento area church says it has been unfair...
645     I am G

# Checking for duplicates and reason for failing detection earlier

In [211]:
a = data['text'][622]

In [212]:
b = data['text'][637]

In [215]:
a = a.split('\n\n')
b = b.split('\n\n')

In [222]:
for i in range(len(a)):
    if a[i]==b[i]:
        continue
    else:
        print(a[i], '**')
        print(b[i], '++')

Dr. Naresh Trehan, Chairman and Managing Director of the Medanta-the Medicity hospital in Gurugram, near New Delhi, said it was vital that authorities know whether a slum has an outbreak. **
Dr. Naresh Trehan, chairman and managing director of the Medanta-the Medicity hospital in Gurugram, near New Delhi, said it was vital that authorities know whether a slum has an outbreak. ++
"This is our darkest hour," said Dr. Rajesh Parikh, Director of Medical Research and Hon. Neuropsychiatrist at Mumbai's Jaslok Hospital and Research Centre. "They build our homes, our factories, our very nation with their bare hands, and now we have forsaken them." **
"This is our darkest hour," said Dr. Rajesh Parikh, director of medical research and Hon. Neuropsychiatrist at Mumbai's Jaslok Hospital and Research Centre. "They build our homes, our factories, our very nation with their bare hands, and now we have forsaken them." ++


Reason detected, above one article contains capitals where as the other doesnt

In [210]:
data['text'][622] == data['text'][637]

False

In [229]:
df['filter'] = df['tags'].apply(lambda x: 1 if 'Galwan Valley' in x else 0)

In [233]:
f_df = df[df['filter']==1].sort_values(by='score', ascending = False).head(10)

In [239]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
def bartsum_store(sents):
    article_input_ids = tokenizer.batch_encode_plus([sents], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
    summary_ids = model.generate(article_input_ids,
                             num_beams=4,
                             length_penalty=2.0,
                             max_length=142,
                             min_len=25,
                             no_repeat_ngram_size=3)

    summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    #display(Markdown('> **abstractive-Summary: **'+summary_txt))
    return summary_txt

In [240]:
f_df['summary'] = f_df['text'].apply(lambda x: bartsum_store(x))

In [241]:
f_df

,_id,channel,query,text,title,author,publish_date,summary,url,image,spacy lang,ldcode,final_l,small_size,tags,score,filter
94,{'$oid': '5f07a5765b177cd19f969f6e'},Google,Line of actual control dispute,Fast News\n\nStand-off between Indian and Chin...,Timeline: the Line of Actual Control between C...,[],{'$date': {'$numberLong': '0'}},"In this study, we look at the factors that det...",https://www.trtworld.com/asia/timeline-the-lin...,https://cdni0.trtworld.com/w480/h270/q75/81741...,"[{'language': 'en', 'score': 0.9999968011502894}]",[en],en,False,"[Valley, stance, India, Galwan Valley, territo...",7.4662,1
102,{'$oid': '5f07a5765b177cd19f969f76'},Google,Galwan dispute,The trajectory of the India-China crisis begin...,Galwan Valley incident: What drove China â€“ a...,[Anurag Viswanath],{'$date': {'$numberLong': '0'}},The India-China border issue is no longer mire...,https://scroll.in/article/965230/galwan-valley...,https://s01.sgp1.cdn.digitaloceanspaces.com/bo...,"[{'language': 'en', 'score': 0.857140146677151...","[[en, 0.8571401466771518], [en, 0.999998382651...",en,False,"[June, India, Valley incident, media, Galwan V...",5.6675,1
0,{'$oid': '5f07a5765b177cd19f969f10'},Google,india china standoff LAC,"India and China, two nuclear-armed Asian neigh...",Five things to know about the India-China bord...,[],{'$date': {'$numberLong': '0'}},Indian and Chinese troops have been engaged in...,https://www.aljazeera.com/news/2020/06/india-c...,https://www.aljazeera.com/mritems/Images/2020/...,"[{'language': 'en', 'score': 0.9999973328097994}]",[en],en,False,"[June, India, Galwan Valley, troops, soldiers,...",5.2087,1
33,{'$oid': '5f07a5765b177cd19f969f31'},Google,india and china border in world,"India and China, two nuclear-armed Asian neigh...",Five things to know about the India-China bord...,[],{'$date': {'$numberLong': '0'}},Indian and Chinese troops have been engaged in...,https://www.aljazeera.com/news/2020/06/india-c...,https://www.aljazeera.com/mritems/Images/2020/...,"[{'language': 'en', 'score': 0.9999968974204423}]",[en],en,False,"[June, India, Galwan Valley, troops, soldiers,...",5.2087,1
55,{'$oid': '5f07a5765b177cd19f969f47'},Google,india and china border in india,"India and China, two nuclear-armed Asian neigh...",Five things to know about the India-China bord...,[],{'$date': {'$numberLong': '0'}},Indian and Chinese troops have been engaged in...,https://www.aljazeera.com/news/2020/06/india-c...,https://www.aljazeera.com/mritems/Images/2020/...,"[{'language': 'en', 'score': 0.9999967027173347}]",[en],en,False,"[June, India, Galwan Valley, troops, soldiers,...",5.2087,1
1,{'$oid': '5f07a5765b177cd19f969f11'},Google,india china standoff LAC,"india\n\nUpdated: Jun 26, 2020 00:22 IST\n\nRe...","LAC standoff: Hope India meets China halfway, ...","[Press Trust Of India, Posted Niyati Singh]",{'$date': {'$numberLong': '1593111092000'}},Chinese Ambassador Sun Weidong said India and ...,https://www.hindustantimes.com/india-news/lac-...,https://www.hindustantimes.com/rf/image_size_9...,"[{'language': 'en', 'score': 0.999994921823019...","[[en, 0.9999949218230191], [nl, 0.999994213254...",en,False,"[sides, June, side, Galwan Valley, consensus]",4.5834,1
112,{'$oid': '5f07a5765b177cd19f969f80'},Google,Galwan dispute,"Without firing a single shot, Indian and Chine...",Why India-China border clash in Galwan Valley ...,"[Marcelo Duhalde, Dennis Wong, Kaliz Lee, Http...",{'$date': {'$numberLong': '0'}},The Galwan Valley contains some of the most tr...,https://multimedia.scmp.com/infographics/news/...,https://multimedia.scmp.com/infographics/news/...,"[{'language': 'en', 'score': 0.9999963180434773}]",[en],en,False,"[sides, Aksai Chin, India, Galwan Valley, Galw...",3.9587,1
108,{'$oid': '5f07a5765b177cd19f969f7c'},Google,Galwan dispute,"The story so far: On June 15, the worst violen...",The Hindu Explains | Who does Galwan Valley be...,"[States Contribute To, Of Active Caseload, He ...",{'$date': {'$numberLong': '0'}},"On June 15, the

In [244]:
f_df[['_id','channel','query','text','title','summary','url']].to_csv('Galwan_Valley_Summaries.csv')